In [ ]:
import ipywidgets as widgets
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    basemaps
)

import datetime as dt
import datacube
import ogr

from utils.DEADataHandling import load_clearsentinel2

In [ ]:
# Load the data
dc = datacube.Datacube(app='agriculture-app')

today = dt.date.today()
start_date = today - dt.timedelta(days=90)  # Calculates the time span available for S2a NRT product

latitude = (-24.974997, -24.995971)
longitude = (152.429994, 152.395805)

time = (start_date.strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d"))  # convert dates to strings

query = {
    'x': longitude,
    'y': latitude,
    'time': time,
    'output_crs': 'EPSG:3577',
    'resolution': (-10,10)
}

measurements = (
    'nbar_red',
    'nbar_green',
    'nbar_blue',
    'nbar_nir_1',
    'nbar_swir_2',
    'nbar_swir_3'
)

ds_s2 = load_clearsentinel2(
    dc=dc,
    query=query,
    sensors=['s2a', 's2b'],
    product='nrt',
    bands_of_interest=measurements,
    masked_prop=0.0  # keep all images, regardless of pixel quality 
)

In [ ]:
zoom = 14

# Construct the object that defines the bounding box of the loaded data

geom_obj = {
    "type": "Feature",
    "properties": {
        "style": {
            "stroke": True,
            "color": 'red',
            "weight": 4,
            "opacity": 0.8,
            "fill": True,
            "fillColor": False,
            "fillOpacity": 0,
            "showArea": True,
            "clickable": True
        }
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    152.395805,
                    -24.995971
                ],
                [
                    152.395805,
                    -24.974997
                ],
                [
                    152.429994,
                    -24.974997
                ],
                [
                    152.429994,
                    -24.995971
                ],
                [
                    152.395805,
                    -24.995971
                ]
            ]
        ]
    }
}

# create a map geometry from the JSON dictionary (It's a python dict right now. Fix and save as a text file external to the notebook)
loadeddata_geometry = ogr.CreateGeometryFromJson(str(geom_obj['geometry']))
center = [loadeddata_geometry.Centroid().GetY(),loadeddata_geometry.Centroid().GetX()]

# define the study area map
studyarea_map = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldImagery)

# define the drawing controls
studyarea_drawctrl = DrawControl(polygon = {"shapeOptions": {"fillOpacity": 0}})

# add drawing controls and data bound geometry to the map
studyarea_map.add_control(studyarea_drawctrl)
studyarea_map.add_layer(GeoJSON(data=geom_obj))

# function that activates after drawing
def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)

# call to say activate handle_draw function on draw
studyarea_drawctrl.on_draw(handle_draw)

# plot the map
studyarea_map